In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import r2_score

import pycaret.regression as pr

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kickstarter/01_df_development.pkl


In [3]:
cols_to_drop = [
    'PROJECT_ID',
    'TRAIN_VAL_TEST_SPLIT',
    'DEADLINE',
    'STATE_CHANGED_AT',
    'CREATED_AT',
    'LAUNCHED_AT',
    'KEYWORDS',
    
    'DESC',
    'NAME',
    'FINAL_STATUS',
]

In [4]:
filepath = fr'/kaggle/input/kickstarter/01_df_development.pkl'
df_development = pd.read_pickle(filepath)
df_development = df_development.drop(cols_to_drop, axis=1)
df_development

,GOAL,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,BACKERS_COUNT,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS
0,60000.0,False,US,USD,4,2375.831389,7.773524,3095.831389,8.038135,720.000000
1,800.0,False,US,USD,41,119.452500,4.791255,1400.657500,7.245411,1281.205000
2,10000.0,False,US,USD,1,6345.470556,8.755654,7185.470556,8.879955,840.000000
3,270.0,False,GB,GBP,0,0.920833,0.652759,192.920833,5.267450,192.000000
4,5.0,False,GB,GBP,0,823.233611,6.714454,1544.233611,7.342930,721.000000
...,...,...,...,...,...,...,...,...,...,...
108124,250.0,False,US,USD,24,835.094444,6.728742,1373.173056,7.225607,538.078611
108125,5000.0,False,US,USD,0,285.817778,5.658847,1005.817778,6.914550,720.000000
108126,45000.0,False,CA,CAD,36,310.938611,5.742806,1030.938611,6.939194,720.000000
108127,3000.0,False,US,USD,0,152.635278,5.034581,873.635278,6.773807,721.000000


In [5]:
s1 = pr.setup(data=df_development, train_size=0.8, target='BACKERS_COUNT', session_id=42)

,Description,Value
0,Session id,42
1,Target,BACKERS_COUNT
2,Target type,Regression
3,Original data shape,"(108129, 10)"
4,Transformed data shape,"(108129, 28)"
5,Transformed train set shape,"(86503, 28)"
6,Transformed test set shape,"(21626, 28)"
7,Numeric features,6
8,Categorical features,2
9,Preprocess,True


In [6]:
def r2_adjusted(y_true, y_pred, **kwargs):
    n = y_true.shape[0]
    p = kwargs['num_predictors'] if 'num_predictors' in kwargs else 1
    r2 = r2_score(y_true, y_pred)
    r2_adj = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    return r2_adj

pr.add_metric('r2_adj', 'R2 Adjusted', r2_adjusted)
pr.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Custom
ID,,,,,,,,
mae,MAE,MAE,<function mean_absolute_error at 0x7af98a18dbd0>,neg_mean_absolute_error,pred,{},False,False
mse,MSE,MSE,<function mean_squared_error at 0x7af98a18dd80>,neg_mean_squared_error,pred,{},False,False
rmse,RMSE,RMSE,<function mean_squared_error at 0x7af98a18dd80>,neg_root_mean_squared_error,pred,{'squared': False},False,False
r2,R2,R2,<function r2_score at 0x7af98a18e050>,r2,pred,{},True,False
rmsle,RMSLE,RMSLE,<function RMSLEMetricContainer.__init__.<local...,"make_scorer(root_mean_squared_log_error, great...",pred,{},False,False
mape,MAPE,MAPE,<function MAPEMetricContainer.__init__.<locals...,"make_scorer(mean_absolute_percentage_error, gr...",pred,{},False,False
r2_adj,R2 Adjusted,R2 Adjusted,<function r2_adjusted at 0x7af9925cc790>,make_scorer(r2_adjusted),pred,{},True,True


In [9]:
best = pr.compare_models(sort='R2 Adjusted', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted,TT (Sec)
lasso,Lasso Regression,166.6788,1517394.5482,1063.2855,0.0058,2.5849,19.1699,0.0056,0.0850
ridge,Ridge Regression,166.4774,1517382.1860,1063.2918,0.0057,2.5748,19.0277,0.0056,0.1020
llar,Lasso Least Angle Regression,166.6789,1517394.5513,1063.2855,0.0058,2.5849,19.1699,0.0056,0.1040
br,Bayesian Ridge,167.2173,1517419.9796,1063.3033,0.0057,2.5949,19.3397,0.0056,0.1060
lr,Linear Regression,166.4927,1517403.9877,1063.3107,0.0057,2.5747,19.0284,0.0055,0.3700
en,Elastic Net,166.2215,1517691.6470,1063.4010,0.0055,2.6032,19.4040,0.0054,0.0930
omp,Orthogonal Matching Pursuit,167.9501,1520968.6705,1065.1187,0.0011,2.7231,21.0518,0.0010,0.1020
dummy,Dummy Regressor,169.1666,1521681.6926,1065.5781,-0.0002,2.7447,21.4324,-0.0003,0.1020
huber,Huber Regressor,118.6626,1532638.3380,1071.4357,-0.0154,1.8854,2.6247,-0.0155,0.0980
gbr,Gradient Boosting Regressor,157.0114,1564577.4747,1099.1551,-0.1369,2.3533,19.4746,-0.1371,6.9200


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [11]:
best_tuned = [pr.tune_model(i, optimize='R2 Adjusted') for i in best]

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
Fold,,,,,,,
0,178.3797,1693982.4387,1301.5308,0.0029,2.6031,19.1203,0.0028
1,162.5798,419553.1173,647.7292,0.0082,2.6278,19.9966,0.0080
2,166.3482,1633462.4544,1278.0698,0.0017,2.6138,19.8964,0.0015
3,158.6068,355740.9164,596.4402,0.0101,2.6502,20.2113,0.0100
4,197.5854,7298993.2602,2701.6649,0.0008,2.6048,19.5111,0.0007
5,159.7848,355801.6094,596.4911,0.0086,2.6301,20.2603,0.0085
6,176.1349,1788035.9322,1337.1746,0.0025,2.6025,19.6677,0.0024
7,160.1705,828277.5086,910.0975,0.0038,2.6263,20.0672,0.0037
8,164.5709,450446.5581,671.1532,0.0074,2.6276,19.6106,0.0073


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
Fold,,,,,,,
0,176.5358,1693453.1973,1301.3275,0.0033,2.5604,18.4058,0.0031
1,161.1647,419245.9339,647.4920,0.0089,2.5875,19.2970,0.0088
2,165.0682,1633454.0511,1278.0665,0.0017,2.5806,19.3312,0.0015
3,157.3080,355457.8064,596.2028,0.0109,2.6134,19.6207,0.0108
4,196.0325,7297836.8863,2701.4509,0.0009,2.5664,18.8527,0.0008
5,158.3712,355805.9077,596.4947,0.0086,2.5926,19.5460,0.0085
6,174.7905,1787692.8328,1337.0463,0.0027,2.5689,19.0874,0.0026
7,158.7918,828031.3228,909.9623,0.0041,2.5896,19.4342,0.0040
8,162.8669,450280.7967,671.0297,0.0078,2.5953,18.9580,0.0077


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
Fold,,,,,,,
0,176.4019,1693478.2925,1301.3371,0.0032,2.5619,18.4468,0.0031
1,161.0318,419174.0054,647.4365,0.0091,2.5888,19.2917,0.0089
2,164.8797,1633472.0962,1278.0736,0.0017,2.5815,19.3162,0.0015
3,157.1376,355456.5421,596.2018,0.0109,2.6147,19.6336,0.0108
4,195.8584,7297914.4098,2701.4652,0.0009,2.5678,18.8663,0.0008
5,158.2437,355773.4962,596.4675,0.0087,2.5939,19.5516,0.0086
6,174.6317,1787733.8998,1337.0617,0.0027,2.5707,19.0919,0.0025
7,158.5769,827965.2583,909.9260,0.0042,2.5902,19.4307,0.0041
8,162.7233,450298.9604,671.0432,0.0078,2.5945,18.9447,0.0077


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [13]:
best_blended = pr.blend_models(best_tuned, optimize='R2 Adjusted')
best_stacked = pr.stack_models(best_tuned, optimize='R2 Adjusted')

best_overall_model = pr.automl(optimize='R2 Adjusted')
best_overall_model

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
Fold,,,,,,,
0,176.3086,1693458.2011,1301.3294,0.0033,2.5595,18.3961,0.0031
1,160.9550,419175.8058,647.4379,0.0091,2.5859,19.2494,0.0089
2,164.8155,1633487.4785,1278.0796,0.0016,2.5789,19.2787,0.0015
3,157.0795,355443.9198,596.1912,0.0109,2.6126,19.5923,0.0108
4,195.7999,7297843.4206,2701.4521,0.0009,2.5646,18.8255,0.0008
5,158.1680,355779.0892,596.4722,0.0087,2.5910,19.5108,0.0086
6,174.5680,1787719.1554,1337.0562,0.0027,2.5687,19.0548,0.0026
7,158.5006,827970.9969,909.9291,0.0042,2.5865,19.3820,0.0041
8,162.6430,450297.1306,671.0418,0.0078,2.5924,18.9077,0.0077


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
Fold,,,,,,,
0,176.2369,1693401.1227,1301.3075,0.0033,2.5526,18.3465,0.0032
1,160.9383,419414.1856,647.6219,0.0085,2.5795,19.2282,0.0084
2,164.7867,1633524.0744,1278.0939,0.0016,2.5711,19.2333,0.0015
3,157.2363,355499.6487,596.2379,0.0108,2.6074,19.5770,0.0107
4,196.7834,7302573.4915,2702.3274,0.0003,2.5586,19.1839,0.0002
5,158.1493,355905.3762,596.5781,0.0083,2.5857,19.4155,0.0082
6,174.4543,1787498.9911,1336.9738,0.0028,2.5611,18.9785,0.0027
7,158.3216,827853.7380,909.8647,0.0044,2.5798,19.2648,0.0042
8,162.5723,450378.8935,671.1027,0.0076,2.5875,18.8423,0.0075


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Lasso(random_state=42)

In [15]:
predict_holdout = pr.predict_model(best_overall_model)
predict_holdout

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2 Adjusted
0,Lasso Regression,169.7118,831052.1793,911.6206,0.0072,2.5779,19.4270,0.0071


,GOAL,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS,BACKERS_COUNT,prediction_label
31731,200.0,False,US,USD,213.207230,5.366944,665.262207,6.501683,452.054993,8,122.387597
34183,100000.0,False,US,USD,19.928888,3.041131,1458.928833,7.286143,1439.000000,1,49.587555
71977,225.0,False,US,USD,7.847778,2.180166,511.847778,6.239979,504.000000,0,24.448565
28759,50000.0,False,US,USD,618.262512,6.428529,2059.262451,7.630589,1441.000000,33,147.139418
50434,5000.0,False,US,USD,614.057495,6.421716,1334.057495,7.196730,720.000000,2,166.991767
...,...,...,...,...,...,...,...,...,...,...,...
94874,8000.0,False,US,USD,431.720276,6.070092,1103.720337,7.007348,672.000000,1,152.829425
50473,75000.0,False,US,USD,49.093056,3.913882,816.027771,6.705673,766.934692,0,80.162374
50943,75000.0,False,GB,GBP,1822.031616,7.508256,2543.031738,7.841506,721.000000,12,192.686040
46601,3975.0,False,US,USD,2406.215332,7.786226,3152.440186,8.056250,746.224976,32,221.747866
